# Procesamiento Digital de Imágenes
## 📸 Clase Final: Generación Automática de Descripciones de Imágenes

En esta última clase vamos a explorar la generación automática de descripciones para imágenes.

Vas a aprender a usar modelos de lenguaje visual (VLMs) para que le pongan un "subtítulo" a tus imágenes, y encima, ¡en español! La idea es que al final de la clase, tengas un sistema funcional que genere estas descripciones automáticamente.

---
### 🎯 Objetivos de Aprendizaje

Al final de esta clase, vas a poder:

*   **Entender cómo funcionan los VLMs (Modelos de Lenguaje-Visión):** Vas a comprender cómo estos modelos combinan lo que "ven" con lo que "entienden" del lenguaje para generar texto.
*   **Generar subtítulos automáticamente:** Vamos a explorar cómo se crean descripciones de imágenes de forma autónoma.
*   **Implementar un sistema completo:** Vas a armar un generador de descripciones de imágenes que funcione en español.

---
### 📌 Conceptos Clave

Para seguir el hilo de la clase, tené en cuenta estos conceptos:

*   **Image Captioning:** Imaginate que tenés una foto y querés que una computadora le ponga un "subtítulo" o una descripción. Bueno, eso es `Image Captioning`. Es el proceso automático de generar texto que describe el contenido visual de una imagen.
*   **BLIP (Bootstrap Language-Image Pre-training):** Este es el modelo de IA que vamos a usar para el `Image Captioning`. Es un modelo de `transformers` que fue entrenado con muchísimas imágenes y textos, lo que le permite entender la relación entre lo visual y lo textual.
*   **Traducción Automática:** Como el modelo BLIP genera las descripciones en inglés, vamos a usar otro modelo para traducir automáticamente esas descripciones al español. Es como tener un traductor instantáneo para nuestro sistema.

In [1]:
# @title Instalación de Dependencias

# Antes de empezar a codear, tenemos que instalar algunas librerías clave que vamos a usar.
# Pensalo como preparar la "caja de herramientas" para nuestro proyecto.

# Vamos a correr estos comandos en tu Google Colab:
!pip install transformers
!pip install gradio
!pip install Pillow
!pip install accelerate -U
!pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

*   `transformers`: Es la librería principal de Hugging Face, nos va a permitir usar modelos de Inteligencia Artificial pre-entrenados para tareas de lenguaje y visión.
*   `gradio`: Con esta librería, vas a poder crear una interfaz web súper sencilla para que cualquiera pueda probar tu modelo sin saber nada de código.
*   `Pillow`: Es la librería de procesamiento de imágenes por excelencia en Python. La vamos a usar para cargar y manipular las fotos.
*   `accelerate`: Ayuda a que los modelos grandes de `transformers` corran más rápido, especialmente si tenés acceso a una GPU.
*   `sentencepiece`: Es una dependencia que necesitan algunos de los modelos de lenguaje para tokenizar el texto.

**Fijate que** una vez que se instalaron todas, Google Colab te va a avisar que *se instalaron correctamente*. Si te pide reiniciar el entorno de ejecución, hacelo.

In [2]:
# @title Importando Librerías y Configuracion de Modelos

import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from transformers import pipeline
import gradio as gr
from PIL import Image

# Configurando Nuestros Cerebros Digitales: Los Modelos
# Ahora que tenemos las librerías instaladas, vamos a cargar los modelos de Inteligencia Artificial que van a hacer la magia.
# Pensá en esto como "descargar" el cerebro que ya fue entrenado para las tareas que necesitamos.

# Mirá el código:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")
traductor = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


**¿Qué estamos haciendo acá?**

1.  **`BlipProcessor` y `BlipForConditionalGeneration`**: Estos son los componentes del modelo BLIP. El `processor` se encarga de preparar la imagen para que el modelo la entienda (por ejemplo, redimensionarla, normalizarla, etc.). El `model` es el que realmente "lee" la imagen y genera el texto. Estamos usando la versión `blip-image-captioning-large`, que es una de las más potentes.
2.  **`pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")`**: Acá estamos usando la función `pipeline` de `transformers` para crear un traductor. Le indicamos que queremos una tarea de `translation` (traducción) y especificamos el modelo `Helsinki-NLP/opus-mt-en-es`, que está entrenado para traducir de inglés (`en`) a español (`es`).

In [3]:
# @title Función `generar_descripcion`

def generar_descripcion(imagen):
    """
    Genera una descripción en español para una imagen dada.
    """
    # Paso 1: Procesar la imagen y generar la descripción en inglés
    # 'processor' prepara la imagen para el modelo (la convierte a un formato que entiende).
    inputs = processor(imagen, return_tensors="pt")

    # 'model.generate' es el corazón del captioning.
    # Le pasamos los inputs procesados y le pedimos que genere hasta 50 palabras.
    out = model.generate(**inputs, max_length=50)

    # 'processor.decode' convierte los números que generó el modelo en texto legible.
    # 'skip_special_tokens=True' saca tokens que no son parte de la descripción.
    descripcion_en = processor.decode(out[0], skip_special_tokens=True)

    # Paso 2: Traducir la descripción al español
    # Usamos nuestro "traductor" para convertir la descripción de inglés a español.
    # [0]['translation_text'] es para extraer el texto de la traducción.
    descripcion_es = traductor(descripcion_en)[0]['translation_text']

    # Paso 3: Devolver los resultados
    # La función devuelve un diccionario con las descripciones en ambos idiomas.
    return {
        "Descripción (English)": descripcion_en,
        "Descripción (Español)": descripcion_es
    }

**¿Qué hace cada línea?**

*   `inputs = processor(imagen, return_tensors="pt")`: Acá, el `processor` toma la `imagen` que le pasás y la convierte a un formato (tensores de PyTorch, indicado por `"pt"`) que el modelo `BLIP` entiende. Es como "preparar" la foto para que el modelo la pueda "leer".
*   `out = model.generate(**inputs, max_length=50)`: Esta es la parte donde el modelo `BLIP` genera la descripción. Le pasamos los `inputs` (la imagen ya procesada) y le decimos `max_length=50` para que la descripción no sea más larga de 50 palabras.
*   `descripcion_en = processor.decode(out[0], skip_special_tokens=True)`: El `model.generate` nos devuelve una secuencia de números (tokens). El `processor.decode` convierte esos números de nuevo en texto legible. Le decimos `skip_special_tokens=True` para que no incluya caracteres raros que usa el modelo internamente. Así obtenemos la descripción en inglés.
*   `descripcion_es = traductor(descripcion_en)[0]['translation_text']`: Acá entra en acción nuestro `traductor`. Le pasamos la `descripcion_en` y él se encarga de traducirla al español. El `[0]['translation_text']` es para extraer el texto de la respuesta del traductor.
*   Finalmente, la función devuelve un diccionario con ambas descripciones, así podés ver tanto la original en inglés como la traducida al español.

In [4]:
# @title 🌐 Interfaz de Usuario

# Para que nuestro sistema sea fácil de usar para cualquiera,
# vamos a crear una interfaz web simple con Gradio.
# Esto nos permite subir una imagen y ver los resultados en el navegador.

# Mirá cómo armamos la interfaz:
with gr.Blocks() as demo:
    gr.Markdown("""
    # 🖼️ Generador Automático de Descripciones de Imágenes
    ¡Subí una imagen y vas a ver cómo el sistema le genera una descripción automáticamente en español!
    """)

    with gr.Row():
        imagen_input = gr.Image(type="pil", label="Subí tu imagen acá")

    with gr.Row():
        boton_analizar = gr.Button("Generar Descripción")

    with gr.Row():
        salida = gr.JSON(label="Resultados")

    boton_analizar.click(
        fn=generar_descripcion,
        inputs=imagen_input,
        outputs=salida
    )

    gr.Markdown("""
    ### 📝 Instrucciones para Usar la Interfaz:
    1.  **Subí una imagen:** Usá el campo de arriba que dice "Subí tu imagen acá". Podés arrastrar y soltar una foto, o **clickear** para seleccionarla desde tu computadora.
    2.  **Generar la descripción:** Una vez que la imagen está cargada, **hacé clic** en el botón "Generar Descripción".
    3.  **Observá los resultados:** Vas a ver un cuadro con formato JSON que te va a mostrar la descripción en inglés y la traducción al español.

    ---
    ### 🤔 Para Pensar un Poco: Reflexiones

    Ahora que probaste el sistema, ¡es momento de ponernos a pensar! Compartí tus ideas y conclusiones.

    *   ¿Qué tan precisas te parecieron las descripciones que generó el modelo? ¿Te sorprendió alguna?
    *   ¿Qué tipo de detalles captó mejor el modelo en las imágenes? ¿Y cuáles no tanto?
    *   Si tuvieras que mejorar algo, ¿cómo creés que se podría mejorar la calidad de las traducciones al español?
    *   Pensando en el mundo real, ¿qué aplicaciones prácticas le ves a esta tecnología? ¿Dónde la usarías?
    """)

# 🚀 ¡Listo para Probarlo!
# Ahora sí, ejecutá esta última celda de código. Se va a abrir una URL (puede que tarde unos segundos).
# Hacé clic en esa URL para abrir la interfaz de Gradio en tu navegador y ¡empezá a probar tu generador de descripciones!
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://da06e4424790763409.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 🏆 Conclusión: ¡Lo Lograste!

¡Felicitaciones! 🎉 Llegaste al final de esta clase y armaste tu propio sistema de generación automática de descripciones de imágenes. Hoy **aprendiste** cómo los Modelos de Lenguaje-Visión (VLMs) pueden "entender" y "describir" el contenido de una imagen, y cómo podés usar la traducción automática para que esos resultados estén en español.

**Viste** cómo se combinan diferentes modelos (uno para el captioning, otro para la traducción) y cómo `Gradio` te permite armar una interfaz de usuario muy rapido.

---
### 👉 Próximos Pasos:

La Inteligencia Artificial es un campo que crece muy rapido. Acá te dejo algunas ideas para que sigas investigando:

*   **Experimentá con otros modelos:** En Hugging Face tenés un montón de modelos de `Image Captioning` y `Traducción` para probar. ¿Hay alguno que funcione mejor?
*   **Mejorá la traducción:** A veces las traducciones automáticas no son perfectas. ¿Cómo podrías incorporar más contexto o incluso usar modelos de traducción más avanzados?
*   **Agregá funcionalidades:** ¿Qué tal si tu sistema pudiera identificar objetos específicos en la imagen o responder preguntas sobre ella? Investigá sobre `Object Detection` o `Visual Question Answering`.
*   **Optimizá el rendimiento:** Si querés que tu modelo sea más rápido, podrías investigar técnicas de cuantización o _distillation_.